In [2]:
import neo4j
import osmnx as ox
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_osm import poidf2rows, aoidf2rows
from utils_neo4j import init, poi_insert_query, aoi_insert_query

In [3]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [4]:
# Define the place you want to search in (e.g., a city name)
place_name = "Hamburg, Germany"  # Or any other place you're interested in

# Get natural areas (parks, lakes, forests, etc.) as GeoDataFrames
tags = {
    'leisure': ['park', 'garden', 'nature_reserve'],
    'natural': ['water', 'wood', 'forest', 'grassland'],
    'landuse': ['forest', 'recreation_ground', 'grass', 'meadow'],
    'water': True,
    'waterway': ['riverbank', 'dock', 'boatyard'],
}


In [5]:
# Fetch the POIs
# Fetch features with polygon geometries
aois = ox.features_from_place(place_name, tags=tags)

In [6]:
aois = aois.reset_index()

In [7]:
mask = aois.name.notna() 
aois = aois[mask]

In [8]:
len(aois)

1001

In [9]:
pd.set_option('display.max_columns', None)
aois.drop(columns=['id']).describe()


,element,geometry,name,waterway,access,description,leisure,seamark:small_craft_facility:category,seamark:type,alt_name,natural,wikidata,wikipedia,canoe,fee,lit,source,surface,survey:date,note,website,garden:type,identity,operator,nudism,addr:postcode,name:en,name:es,name:pt,addr:city,addr:housenumber,addr:street,fax,phone,leaf_type,leaf_cycle,amenity,water,height,species,drinking_water,genus,species:de,start_date,email,seamark:name,tidal,building,dock,maxdraught,maxlength,maxwidth,boat,motorboat,ship,intermittent,bicycle,landuse,area,barrier,fence_type,opening_hours,sport,depth,description:de,description:en,salt,contact:website,basin,check_date,ele,charge,covered,maxheight,maxstay,park_ride,parking,smoothness,supervised,seamark:harbour:category,short_name,island,heritage,heritage:operator,ref:dsa,wheelchair,image,created_by,fishing,communication:amateur_radio:pota,reservoir_type,ref,layer,farm:type,name:source,cemetery:historic,url,loc_name,boundary,protect_class,protection_title,maxspeed,seamark:restricted_area:restriction,TMC:cid_58:tabcd_1:Class,TMC:cid_58:tabcd_1:LCLversion,TMC:cid_58:tabcd_1:LocationCode,architect,dsa:criteria,seasonal,club,community_centre,emergency,type,official_name,disused:landuse,note:de,name:etymology:wikidata,playground:theme,long_name,addr:country,check_date:opening_hours,community_centre:for,internet_access,name:etymology,old_name,fixme,farmland,dog,network,toilets:wheelchair,tourism,place,local_name,archaeological_site,historic,noname,contact:email,contact:phone,operator:type,wetland,disused,landcover,addr:suburb,disused:lock,lock,wheelchair:description,ref:sh-nsg,area:highway,source:name,meadow,was:landuse,was:name,harbour,capacity,trees,short_protection_title,man_made,operator:wikidata,kerb,info,wikimedia_commons,animal_training,industrial,maxdraft,comment,garden:style,payment:cash,payment:credit_cards,payment:cryptocurrencies,payment:electronic_purses,payment:girocard,theatre:type,construction,source:geometry,playground,playground:springy,bottle,material,level,source:website,proposed,nohousenumber,hoops,traffic_calming,ref:WDPA,status,boatyard:type,artist,source:date,fenced,postal_code,aeroway,note:mapping_status,protection_object,name:ru,protection_ban,CEMT,winter_service,year_of_construction,name:etymology:wikipedia
count,1001,1001,1001,17,129,21,526,2,19,17,308,170,94,5,10,2,27,6,0,9,49,11,1,147,2,10,1,1,1,9,10,11,1,8,24,13,8,142,1,0,0,0,0,24,6,9,38,8,8,8,8,8,17,10,10,24,1,301,0,12,1,17,8,1,2,1,24,5,18,20,1,1,1,1,1,1,1,1,4,6,37,0,11,11,11,30,9,0,6,23,0,19,1,2,5,3,5,4,49,50,48,12,6,3,3,3,4,6,1,1,1,0,180,2,2,2,5,0,36,3,2,1,1,2,3,4,0,4,1,3,5,1,0,1,4,0,1,1,4,0,1,1,1,0,0,1,3,0,1,3,0,0,1,0,0,4,0,1,0,1,6,1,1,0,0,4,1,1,1,1,1,1,0,0,0,0,0,1,0,0,1,1,0,0,2,2,1,1,2,0,1,1,0,1,1,2,1,1,1,2
unique,3,991,877,2,6,21,11,1,5,17,4,162,88,1,2,2,23,2,0,9,48,6,1,33,1,9,1,1,1,1,10,10,1,8,3,3,7,9,1,0,0,0,0,20,6,9,2,2,2,6,8,8,4,3,1,2,1,10,0,2,1,13,2,1,2,1,1,5,2,13,1,1,1,1,1,1,1,1,1,2,32,0,1,1,11,3,9,0,3,21,0,17,1,2,2,1,5,4,1,3,4,2,1,1,1,3,4,1,1,1,1,0,2,2,1,1,5,0,31,1,2,1,1,2,3,4,0,2,1,2,2,1,0,1,3,0,1,1,2,0,1,1,1,0,0,1,3,0,1,2,0,0,1,0,0,2,0,1,0,1,6,1,1,0,0,4,1,1,1,1,1,1,0,0,0,0,0,1,0,0,1,1,0,0,2,1,1,1,1,0,1,1,0,1,1,1,1,1,1,2
top,way,"POLYGON ((10.0334232 53.7208519, 10.0335525 53...",Brack,boatyard,private,Urban Gardening des KulturEnergieBunkerAltonaP...,park,boatyard,harbour,Kleiner Bramfelder See,water,Q2548173,de:Wandsbeker Gehölz,yes,no,yes,"yahoo, accurate",grass,NaN,A place for Urban Gardening.,https://www.hamburg.de/duvenstedter-brook/,community,commons,Klgv. 578,yes,22301,Japanese Garden,Jardín Japonés,Jardim Japonês,Hamburg,38d,Poßmoorweg,+49 40 4801870,+49 40 487824,mixed,deciduous,fountain,pond,2,NaN,NaN,NaN,NaN,1981,Peter.Knief@hamburg.de,Knief-Bootswerft,yes,dock,floating,8.0 m,320.0 m,52.0 m,yes,yes,yes,no,yes,orchard,NaN,fence,wire,24/7,swimming,30,Schwimmen verboten.,Swimming is not allowed.,no,http://www.alster-bille-elbe-parks.hamburg/,retention,2024-03-06,2.8,2 eur,no,none,12 hours,yes,surface,good

In [10]:
# description based on: 'description:de', 'description' 
# tags based on: 'amenity', 'bar', 'branch', 'brand', 'cuisine'
tagcols = ['leisure', 'access', 'waterway', 'natural', 'water', 'landuse', 'protection_title']
impcols = ['id', 'geometry', 'name', 'addr:street', 'addr:housenumber', 'addr:postcode', 'description:de', 'description', 'note', 'wikipedia', *tagcols]

In [11]:
aois_impcols = aois[impcols]

In [12]:
mask = aois_impcols.name.isna()
aois_impcols = aois_impcols[~mask]

In [13]:
aois_impcols['geometry'] = aois_impcols['geometry'].apply(simplify_to_centroid_if_small)

In [14]:
# step 1
# combine col description:de and description, use description:de 
# if available else description, if both are not available use empty string
aois_impcols['description'] = aois_impcols['description:de'].combine_first(aois_impcols['description'])
aois_impcols['description'] = aois_impcols['note'].combine_first(aois_impcols['description'])
aois_impcols['description'] = aois_impcols['wikipedia'].combine_first(aois_impcols['description'])
aois_impcols.drop(columns=['description:de', 'note', 'wikipedia'], inplace=True)

In [15]:
# step 2
# 'amenity', 'bar', 'branch', 'cuisine'
# put values in a col called tags which is list of strings but only if the value is not null or empty string
# remove the columns from aois_impcols
# Modified code to add column name prefixes to tag values
aois_impcols['tags'] = aois_impcols[tagcols].apply(
    lambda row: [f"{col}:{val}" for col, val in zip(row.index, row) 
                if pd.notna(val) and val != ''], 
    axis=1
)
aois_impcols.drop(columns=tagcols, inplace=True)

In [16]:
# drop columns where geometry is empty 
aois_impcols = aois_impcols[aois_impcols['geometry'].notna()]
# drop columns where name and description and tags is empty
mask = aois_impcols.tags.apply(lambda x: len(x) > 0) | aois_impcols.name.notna() | aois_impcols.description.notna()
aois_impcols = aois_impcols[mask]

In [17]:
# rename col id to osmid
aois_impcols.rename(columns={'id': 'osmid'}, inplace=True)

In [18]:
aois_impcols.head()

,osmid,geometry,name,addr:street,addr:housenumber,addr:postcode,description,tags
0,256489803,POINT (9.76468 53.56729),Sven-Simon-Park,NaN,NaN,NaN,NaN,[leisure:park]
1,3623157837,POINT (10.05191 53.60005),FuhlsGarden,NaN,NaN,NaN,A place for Urban Gardening.,[leisure:park]
2,3739293405,POINT (9.9519 53.55183),KEBAPgarten am Bunker,NaN,NaN,NaN,Urban Gardening des KulturEnergieBunkerAltonaP...,[leisure:garden]
3,4369802587,POINT (10.13482 53.5126),Boberger See - FKK Liegewiese,NaN,NaN,NaN,NaN,[leisure:park]
4,4369802588,POINT (10.13776 53.51531),Liegewiese Boberger See,NaN,NaN,NaN,NaN,[leisure:park]


In [19]:
# filter rows where geometry dont is point
pois_impcols = aois_impcols[aois_impcols['geometry'].apply(lambda x: x.geom_type == 'Point')]
aois_impcols = aois_impcols[aois_impcols['geometry'].apply(lambda x: x.geom_type != 'Point')]

In [20]:
len(pois_impcols)

391

In [21]:
poi_rows = poidf2rows(pois_impcols)

In [22]:
poi_rows

[{'osmid': 256489803,
  'name': 'Sven-Simon-Park',
  'addr_street': None,
  'addr_housenumber': None,
  'addr_postcode': None,
  'description': None,
  'tags': ['leisure:park'],
  'longitude': 9.7646796,
  'latitude': 53.5672937},
 {'osmid': 3623157837,
  'name': 'FuhlsGarden',
  'addr_street': None,
  'addr_housenumber': None,
  'addr_postcode': None,
  'description': 'A place for Urban Gardening.',
  'tags': ['leisure:park'],
  'longitude': 10.0519079,
  'latitude': 53.6000474},
 {'osmid': 3739293405,
  'name': 'KEBAPgarten am Bunker',
  'addr_street': None,
  'addr_housenumber': None,
  'addr_postcode': None,
  'description': 'Urban Gardening des KulturEnergieBunkerAltonaProjekt (KEBAP) e.V.',
  'tags': ['leisure:garden'],
  'longitude': 9.9518976,
  'latitude': 53.5518336},
 {'osmid': 4369802587,
  'name': 'Boberger See - FKK Liegewiese',
  'addr_street': None,
  'addr_housenumber': None,
  'addr_postcode': None,
  'description': None,
  'tags': ['leisure:park'],
  'longitude': 10.

In [23]:
# Execute the query
with driver.session() as session:
    result = session.run(poi_insert_query, rows=poi_rows)
    print(f"Imported {result.single()['total']} pois")



Imported 391 pois


In [24]:
aoi_rows = aoidf2rows(aois_impcols)

In [25]:
aoi_rows

[{'osmid': 10843,
  'name': 'Stadtparksee',
  'description': 'de:Stadtparksee',
  'tags': ['natural:water', 'water:pond'],
  'geometry_wkt': 'POLYGON ((10.0250794 53.5947103, 10.0253203 53.5948048, 10.0255479 53.594848, 10.0257816 53.5948748, 10.0259718 53.5948771, 10.0261357 53.5948595, 10.0263142 53.5948225, 10.0264198 53.5948015, 10.0264888 53.5947842, 10.0265592 53.5947665, 10.0268111 53.5946927, 10.026996 53.5945982, 10.0271408 53.5945019, 10.027237 53.5943488, 10.0273286 53.5942082, 10.0274052 53.5940854, 10.0274031 53.5939291, 10.027364 53.5937563, 10.027352 53.5936995, 10.0273831 53.5936661, 10.0272764 53.5935075, 10.027306 53.5935005, 10.0272314 53.5933897, 10.0272749 53.5933787, 10.0265326 53.5923224, 10.0264884 53.5923326, 10.0264113 53.5922213, 10.0263745 53.5922303, 10.0263192 53.5921575, 10.0261863 53.5920743, 10.0260044 53.5919222, 10.0259024 53.5918285, 10.0257881 53.5917688, 10.0256899 53.5917026, 10.0255847 53.5916627, 10.0254816 53.5916432, 10.0254537 53.5916212, 10.

In [ ]:
# Execute the query
with driver.session() as session:
    result = session.run(aoi_insert_query, rows=aoi_rows)
    print(f"Imported {result.single()['total']} aois")

driver.close()

Imported 610 pois
